# Démo Micro-Robot avec PID

Ce notebook montre :
- comment définir les paramètres du système (`I`, `k`, `c`),
- comment appliquer différents contrôleurs (`zero_control`, `sinus_control`, `PID`),
- comment ajuster les gains PID et les paramètres physiques en direct avec des sliders interactifs.

In [1]:
from ipywidgets import FloatSlider, interactive, VBox
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from src.simulation import run_simulation
from src.control import PID

## Démo interactive (PID + paramètres système)

In [2]:
def simulate(I, k, c, Kp, Ki, Kd, target, t_max):
    params = {"I": I, "k": k, "c": c}
    pid = PID(Kp=Kp, Ki=Ki, Kd=Kd, target=target)

    # Simulation
    t, y = run_simulation(params, control_func=pid, t_max=t_max)

    # Calcul de la commande
    u_vals = [pid(ti, yi) for ti, yi in zip(t, y.T)]

    fig, ax1 = plt.subplots(figsize=(6, 4))

    # Angle simulé
    ax1.plot(t, y[0], label="Angle θ (rad)", color="tab:blue")
    # Consigne constante
    ax1.plot(t, [target]*len(t), "g--", label="Consigne θ*")

    ax1.set_xlabel("Temps (s)")
    ax1.set_ylabel("Angle (rad)", color="tab:blue")
    ax1.tick_params(axis="y", labelcolor="tab:blue")

    # Commande PID
    ax2 = ax1.twinx()
    ax2.plot(t, u_vals, label="Commande u(t)", color="tab:red", linestyle="--")
    ax2.set_ylabel("Commande (u)", color="tab:red")
    ax2.tick_params(axis="y", labelcolor="tab:red")

    # Légendes
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")

    plt.title(f"PID: Kp={Kp}, Ki={Ki}, Kd={Kd}, cible={target}")
    fig.tight_layout()
    plt.show()


# --- Sliders ---
ui = VBox([
    FloatSlider(value=1e-4, min=1e-6, max=1e-2, step=1e-6, description="Inertie I", readout_format=".1e"),
    FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description="Raideur k"),
    FloatSlider(value=0.05, min=0.0, max=1.0, step=0.01, description="Amort. c"),
    FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description="Kp"),
    FloatSlider(value=0.1, min=0.0, max=5.0, step=0.1, description="Ki"),
    FloatSlider(value=0.05, min=0.0, max=2.0, step=0.05, description="Kd"),
    FloatSlider(value=0.0, min=-2.0, max=2.0, step=0.1, description="Cible θ*"),
    FloatSlider(value=5, min=2, max=20, step=1, description="Durée (s)")
])

out = interactive(
    simulate,
    I=ui.children[0], k=ui.children[1], c=ui.children[2],
    Kp=ui.children[3], Ki=ui.children[4], Kd=ui.children[5],
    target=ui.children[6], t_max=ui.children[7]
)

display(out)


interactive(children=(FloatSlider(value=0.0001, description='Inertie I', max=0.01, min=1e-06, readout_format='…